In [1]:
import os
import sys
# Add project root to sys.path for importing from src
# This block includes robust path handling for notebook environments
try:
    # Attempt to get the script directory if __file__ is defined
    # Assuming the script is within the project structure (like in 'notebooks' or 'src')
    script_dir = os.path.dirname(os.path.abspath(__file__))
    # Project root is parent of script directory
    project_root = os.path.abspath(os.path.join(script_dir, os.pardir))

except NameError:
    # If __file__ is not defined, use the current working directory
    # Assume current working directory is within the project structure (like 'notebooks')
    current_working_dir = os.getcwd()
    # Project root is parent of current working directory
    project_root = os.path.abspath(os.path.join(current_working_dir, os.pardir))
    # Optional: Add a message box for debugging - requires root window later
    # messagebox.showwarning("Environment Warning", f"Could not determine script directory using __file__. Assuming project root based on CWD: '{project_root}'.")


# Add the project root to sys.path so imports from src work
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added project root to sys.path: {project_root}") # Optional print


import tkinter as tk
from tkinter import ttk, filedialog, messagebox, scrolledtext
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from mpl_toolkits.mplot3d import Axes3D # Needed for 3D plots


# Import all necessary classes from your src directory
# Make sure these files (data_loader.py, preprocessing.py, dimensionality_reduction.py, evaluation.py) exist and contain the correct classes
try:
    from src.data_loader import DataLoader
    from src.preprocessing import DataPreprocessor
    # Import all DR algorithms you have implemented, including the new ones
    from src.dimensionality_reduction import GeneticAlgorithmDR, PCADR, DifferentialEvolutionDR, MemeticAlgorithmDR, SOMDR, HybridGASOMDR
    from src.evaluation import ModelEvaluator

    # Import necessary models for evaluation
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier # Example

    # Import MiniSom if it's used directly in the GUI (though it's mostly in DR classes now)
    # from minisom import MiniSom # Uncomment if needed directly

except ImportError as e:
    # This error check now happens after setting sys.path
    messagebox.showerror("Import Error", f"Could not import required modules. Ensure 'src' directory is in your Python path ({project_root}) and contains all necessary files.\nError: {e}")
    # You might want to exit the application if core modules fail to import
    # root.destroy() # root not available here yet
    # exit()
    pass # Allow the app to start but with errors if in notebook

class DRGuiApp:
    def __init__(self, root):
        self.root = root
        root.title("Dimensionality Reduction Pipeline")
        root.geometry("1200x900") # Adjusted size to fit more options and plot

        # --- Variables to hold data and parameters ---
        self.target_column = tk.StringVar(value='Diabetes_binary')
        self.test_size = tk.DoubleVar(value=0.2)
        self.random_state = tk.IntVar(value=42)
        self.target_dim = tk.IntVar(value=3) # Target output dimension for DR

        # GA Parameters
        self.ga_pop_size = tk.IntVar(value=50)
        self.ga_n_gen = tk.IntVar(value=50)
        self.ga_mut_prob = tk.DoubleVar(value=0.1)
        self.ga_cx_prob = tk.DoubleVar(value=0.7)
        self.ga_k_tournament = tk.IntVar(value=5)

        # DE Parameters
        self.de_pop_size = tk.IntVar(value=30)
        self.de_n_gen = tk.IntVar(value=30)
        self.de_F = tk.DoubleVar(value=0.5)
        self.de_CR = tk.DoubleVar(value=0.9)

        # MA Parameters
        self.ma_pop_size = tk.IntVar(value=50)
        self.ma_n_gen = tk.IntVar(value=50)
        self.ma_mut_prob = tk.DoubleVar(value=0.1)
        self.ma_cx_prob = tk.DoubleVar(value=0.7)
        self.ma_k_tournament = tk.IntVar(value=5)
        self.ma_local_search_prob = tk.DoubleVar(value=0.3)

        # SOM Parameters
        # Grid dimensions - User will enter these based on desired output dimensions (e.g., 10x10x10 for 3D)
        self.som_dim1 = tk.IntVar(value=10)
        self.som_dim2 = tk.IntVar(value=10)
        self.som_dim3 = tk.IntVar(value=10) # Defaulting to 10x10x10 grid as a common example
        self.som_sigma = tk.DoubleVar(value=1.0)
        self.som_learning_rate = tk.DoubleVar(value=0.5)
        self.som_iterations = tk.IntVar(value=1000)

        # Hybrid GA+SOM Parameters (New)
        self.hybrid_intermediate_dim = tk.IntVar(value=10) # Intermediate linear projection dimension
        self.hybrid_som_dim1 = tk.IntVar(value=10)        # SOM grid dim 1
        self.hybrid_som_dim2 = tk.IntVar(value=10)        # SOM grid dim 2
        self.hybrid_som_dim3 = tk.IntVar(value=1)         # SOM grid dim 3 (defaulting to 1 for potential 2D SOM output)
        self.hybrid_som_sigma = tk.DoubleVar(value=1.0)
        self.hybrid_som_learning_rate = tk.DoubleVar(value=0.5)
        self.hybrid_som_iterations_eval = tk.IntVar(value=500) # SOM iterations during GA evaluation
        self.hybrid_som_iterations_final = tk.IntVar(value=1000) # SOM iterations for final model
        # Reusing standard GA parameters for hybrid's GA part for simplicity in GUI.
        # If you need separate parameters, define them here and update _setup_hybrid_params and _run_dr.
        self.hybrid_ga_pop_size = tk.IntVar(value=50) # Using new vars for clarity
        self.hybrid_ga_n_gen = tk.IntVar(value=50)
        self.hybrid_ga_mut_prob = tk.DoubleVar(value=0.1)
        self.hybrid_ga_cx_prob = tk.DoubleVar(value=0.7)
        self.hybrid_ga_k_tournament = tk.IntVar(value=5)


        # Evaluation Parameters
        self.eval_model_name = tk.StringVar(value="Logistic Regression")
        # Parameters for specific evaluation models
        self.lr_max_iter = tk.IntVar(value=2000)
        self.lr_solver = tk.StringVar(value='liblinear')
        # Add variables for other evaluation models if needed (e.g., rf_n_estimators)
        self.rf_n_estimators = tk.IntVar(value=100)
        self.rf_max_depth = tk.IntVar(value=5)
        # --- Data storage ---
        self.df = None
        self.X_train_scaled = None
        self.X_test_scaled = None
        self.y_train = None
        self.y_test = None
        self.feature_names = None
        self.X_all_scaled = None # Full scaled data (for visualization)
        self.y_all_viz = None # Full target data (for visualization)

        # --- DR Results Storage ---
        # Store the DR model instances
        self.ga_dr_model = None
        self.pca_dr_model = None
        self.de_dr_model = None
        self.ma_dr_model = None
        self.som_dr_model = None
        self.hybrid_dr_model = None # Store Hybrid GA+SOM model

        # Store the projected data for visualization (using the full scaled dataset)
        # These will be the shape (n_samples, target_dim) or (n_samples, len(grid_dims) for SOM/Hybrid)
        self.Z_best_ga_viz = None
        self.X_pca_viz = None
        self.Z_best_de_viz = None
        self.Z_best_ma_viz = None
        self.Z_som_viz = None
        self.Z_best_hybrid_viz = None # Store Hybrid GA+SOM projected data for visualization

        # Store the accuracy results (evaluated using the model from the Evaluation tab)
        self.ga_accuracy = None
        self.pca_accuracy = None
        self.de_accuracy = None
        self.ma_accuracy = None
        self.som_accuracy = None
        self.hybrid_accuracy = None # Store Hybrid GA+SOM accuracy


        # --- GUI Widgets ---
        self._create_widgets()


    def _create_widgets(self):
        """Creates the main widgets and layout using tabs."""
        notebook = ttk.Notebook(self.root)
        notebook.pack(pady=10, padx=10, fill='both', expand=True)

        # --- Data Loading Tab ---
        data_frame = ttk.Frame(notebook, padding="10")
        notebook.add(data_frame, text='Data Loading')
        self._setup_data_loading_tab(data_frame)

        # --- Preprocessing Tab ---
        preprocessing_frame = ttk.Frame(notebook, padding="10")
        notebook.add(preprocessing_frame, text='Preprocessing')
        self._setup_preprocessing_tab(preprocessing_frame)

        # --- Dimensionality Reduction Tab ---
        dr_frame = ttk.Frame(notebook, padding="10")
        notebook.add(dr_frame, text='Dimensionality Reduction')
        self._setup_dimensionality_reduction_tab(dr_frame)

        # --- Evaluation Tab ---
        evaluation_frame = ttk.Frame(notebook, padding="10")
        notebook.add(evaluation_frame, text='Evaluation')
        self._setup_evaluation_tab(evaluation_frame)

        # --- Visualization Tab ---
        visualization_frame = ttk.Frame(notebook, padding="10")
        notebook.add(visualization_frame, text='Visualization')
        self._setup_visualization_tab(visualization_frame)

        # Automatically attempt to load data on startup
        self._load_data_automatically()


    # --- Data Loading Tab Methods ---
    def _setup_data_loading_tab(self, frame):
        """Sets up widgets for the Data Loading tab."""
        ttk.Label(frame, text="Loading data from fixed location: data/diabetes_binary_5050split_health_indicators_BRFSS2015.csv").grid(row=0, column=0, columnspan=3, sticky='w', pady=5, padx=5)
        ttk.Label(frame, text="Target Column Name:").grid(row=1, column=0, sticky='w', pady=5, padx=5)
        ttk.Entry(frame, textvariable=self.target_column, width=30).grid(row=1, column=1, sticky='w', pady=5, padx=5)
        self.data_info_text = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=70, height=10)
        self.data_info_text.grid(row=3, column=1, columnspan=2, pady=5, padx=5, sticky='nsew')
        ttk.Label(frame, text="Data Info:").grid(row=3, column=0, sticky='nw', pady=5, padx=5)
        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(3, weight=1)


    def _load_data_automatically(self):
        """Loads the data automatically from the predefined path."""
        data_file_name = 'diabetes_binary_5050split_health_indicators_BRFSS2015.csv'
        data_path = None # Variable to store the determined data path

        try:
            # Attempt to get the script directory if __file__ is defined
            # Assuming the script is within the project structure (like in 'notebooks' or 'src')
            script_dir = os.path.dirname(os.path.abspath(__file__))
            # Construct path assuming 'data' is a sibling of the script's parent directory
            # e.g., if script is in notebooks, parent is ci-project, data is sibling of notebooks
            project_root_from_script = os.path.abspath(os.path.join(script_dir, os.pardir))
            data_path = os.path.join(project_root_from_script, 'data', data_file_name)


        except NameError:
            # If __file__ is not defined (e.g., in interactive environments), use the current working directory
            # Assume current working directory is within the project structure (like 'notebooks')
            current_working_dir = os.getcwd()
            # Construct path assuming 'data' is a sibling of the current working directory's parent
            # e.g., if cwd is notebooks, parent is ci-project, data is sibling of notebooks
            project_root_from_cwd = os.path.abspath(os.path.join(current_working_dir, os.pardir))
            data_path = os.path.join(project_root_from_cwd, 'data', data_file_name)

            # Optional: Add a message box to inform the user about the fallback for debugging
            # messagebox.showwarning("Environment Warning", f"Could not determine script directory using __file__. Using current working directory '{current_working_dir}' for relative data path.")


        target = self.target_column.get()

        if not target:
            messagebox.showwarning("Input Error", "Target column name is not set.")
            self.data_info_text.delete('1.0', tk.END)
            self.data_info_text.insert(tk.END, "Data Info: Target column name missing.")
            return

        self.data_info_text.delete('1.0', tk.END)
        self.data_info_text.insert(tk.END, f"Attempting to load data from: {data_path}\n")
        self.data_info_text.insert(tk.END, f"Using target column: {target}\n\n")

        loaded = False
        try:
            if os.path.exists(data_path):
                data_loader = DataLoader(data_path)
                self.df = data_loader.load_data()
                loaded = True
                self.data_info_text.insert(tk.END, f"Successfully loaded from: {data_path}\n\n")
            else:
                raise FileNotFoundError(f"Data file not found at expected path: {data_path}")

            # Display data info if loaded
            if loaded:
                self.data_info_text.insert(tk.END, f"Data loaded successfully. Shape: {self.df.shape}\n\n")
                self.data_info_text.insert(tk.END, "Columns:\n")
                self.data_info_text.insert(tk.END, "\n".join(self.df.columns) + "\n\n")
                self.data_info_text.insert(tk.END, "Data Head:\n")
                self.data_info_text.insert(tk.END, self.df.head().to_string())

                messagebox.showinfo("Success", "Data loaded automatically!")

        except FileNotFoundError as e:
            messagebox.showerror("Loading Error", str(e))
            self._reset_data()
        except Exception as e:
            messagebox.showerror("Loading Error", f"An error occurred during data loading: {e}")
            self._reset_data()

    def _reset_data(self):
        """Resets data related variables on error."""
        self.df = None
        self.data_info_text.delete('1.0', tk.END)
        self.data_info_text.insert(tk.END, "Data Info: Failed to load data.")
        self._reset_preprocessing_results()


    # --- Preprocessing Tab Methods ---
    def _setup_preprocessing_tab(self, frame):
        """Sets up widgets for the Preprocessing tab."""
        ttk.Label(frame, text="Test Size:").grid(row=0, column=0, sticky='w', pady=5, padx=5)
        ttk.Entry(frame, textvariable=self.test_size, width=10).grid(row=0, column=1, sticky='w', pady=5, padx=5)

        ttk.Label(frame, text="Random State:").grid(row=1, column=0, sticky='w', pady=5, padx=5)
        ttk.Entry(frame, textvariable=self.random_state, width=10).grid(row=1, column=1, sticky='w', pady=5, padx=5)

        ttk.Button(frame, text="Preprocess Data", command=self._preprocess_data).grid(row=2, column=0, columnspan=2, pady=10)

        ttk.Label(frame, text="Preprocessing Info:").grid(row=3, column=0, sticky='nw', pady=5, padx=5)
        self.preprocessing_info_text = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=70, height=8)
        self.preprocessing_info_text.grid(row=3, column=1, columnspan=2, pady=5, padx=5, sticky='nsew')

        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(3, weight=1)
    def _preprocess_data(self):
         if self.df is None:
             messagebox.showwarning("Preprocessing Error", "Please load data first.")
             return
         if not self.target_column.get():
              messagebox.showwarning("Preprocessing Error", "Target column name is not set.")
              return
         try:
             preprocessor = DataPreprocessor(
                 target_column=self.target_column.get(),
                 test_size=self.test_size.get(),
                 random_state=self.random_state.get()
             )
             self.X_train_scaled, self.X_test_scaled, self.y_train, self.y_test, self.feature_names = preprocessor.preprocess(self.df)
             self.X_all_scaled = preprocessor.scaler.transform(preprocessor.X) # Transform the full dataset for visualization
             self.y_all_viz = preprocessor.y # Keep original y for visualization coloring

             self.preprocessing_info_text.delete('1.0', tk.END)
             self.preprocessing_info_text.insert(tk.END, "Data preprocessed successfully.\n\n")
             self.preprocessing_info_text.insert(tk.END, f"X_train_scaled shape: {self.X_train_scaled.shape}\n")
             self.preprocessing_info_text.insert(tk.END, f"X_test_scaled shape: {self.X_test_scaled.shape}\n")
             self.preprocessing_info_text.insert(tk.END, f"y_train shape: {self.y_train.shape}\n")
             self.preprocessing_info_text.insert(tk.END, f"y_test shape: {self.y_test.shape}\n")
             self.preprocessing_info_text.insert(tk.END, f"Original number of features: {len(self.feature_names)}\n")
             messagebox.showinfo("Success", "Data preprocessed successfully!")
         except ValueError as e:
              messagebox.showerror("Preprocessing Error", f"Input error during preprocessing: {e}")
              self._reset_preprocessing_results()
         except Exception as e:
             messagebox.showerror("Preprocessing Error", f"An error occurred during data preprocessing: {e}")
             self._reset_preprocessing_results()
             
    def _reset_preprocessing_results(self):
         """Resets preprocessing results and subsequent step results."""
         self.X_train_scaled = None
         self.X_test_scaled = None
         self.y_train = None
         self.y_test = None
         self.feature_names = None
         self.X_all_scaled = None
         self.y_all_viz = None
         self.preprocessing_info_text.delete('1.0', tk.END)
         self.preprocessing_info_text.insert(tk.END, "Preprocessing Info: Results reset or failed.")
         self._reset_dr_results()


    # --- Dimensionality Reduction Tab Methods ---
    def _setup_dimensionality_reduction_tab(self, frame):
        """Sets up widgets for the Dimensionality Reduction tab."""
        ttk.Label(frame, text="Target Dimension:").grid(row=0, column=0, sticky='w', pady=5, padx=5)
        ttk.Entry(frame, textvariable=self.target_dim, width=10).grid(row=0, column=1, sticky='w', pady=5, padx=5)

        ttk.Label(frame, text="Select Algorithm:").grid(row=1, column=0, sticky='w', pady=5, padx=5)
        self.dr_algorithm_var = tk.StringVar(value="Genetic Algorithm") # Default selection
        # Add all algorithm radio buttons here. Adjust column number based on how many you have.
        # Assuming 6 algorithms: PCA, GA, DE, MA, SOM, Hybrid -> requires columns 1 through 6 for radio buttons
        ttk.Radiobutton(frame, text="PCA", variable=self.dr_algorithm_var, value="PCA", command=self._toggle_dr_params).grid(row=1, column=1, sticky='w', pady=5, padx=5)
        ttk.Radiobutton(frame, text="Genetic Algorithm", variable=self.dr_algorithm_var, value="Genetic Algorithm", command=self._toggle_dr_params).grid(row=1, column=2, sticky='w', pady=5, padx=5)
        ttk.Radiobutton(frame, text="Differential Evolution", variable=self.dr_algorithm_var, value="Differential Evolution", command=self._toggle_dr_params).grid(row=1, column=3, sticky='w', pady=5, padx=5)
        ttk.Radiobutton(frame, text="Memetic Algorithm", variable=self.dr_algorithm_var, value="Memetic Algorithm", command=self._toggle_dr_params).grid(row=1, column=4, sticky='w', pady=5, padx=5)
        ttk.Radiobutton(frame, text="Self-Organizing Map", variable=self.dr_algorithm_var, value="Self-Organizing Map", command=self._toggle_dr_params).grid(row=1, column=5, sticky='w', pady=5, padx=5)
        ttk.Radiobutton(frame, text="Hybrid GA+SOM", variable=self.dr_algorithm_var, value="Hybrid GA+SOM", command=self._toggle_dr_params).grid(row=1, column=6, sticky='w', pady=5, padx=5) # Added Hybrid


        # --- Parameter Sections ---
        # Create label frames for each algorithm's parameters.
        # Position them all at row 2, column 0 with a columnspan covering all algorithm columns + label column.
        # Their visibility will be controlled by _toggle_dr_params.
        total_algorithm_columns = 6 # Number of algorithm radio buttons
        label_column_span = 1       # Column for the "Select Algorithm:" label
        total_colspan = total_algorithm_columns + label_column_span # Total columns spanned by params frame

        self.pca_params_frame = ttk.LabelFrame(frame, text="PCA Parameters", padding="10")
        self.pca_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        self._setup_pca_params(self.pca_params_frame)

        self.ga_params_frame = ttk.LabelFrame(frame, text="Genetic Algorithm Parameters", padding="10")
        self.ga_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        self._setup_ga_params(self.ga_params_frame)

        self.de_params_frame = ttk.LabelFrame(frame, text="Differential Evolution Parameters", padding="10")
        self.de_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        self._setup_de_params(self.de_params_frame)

        self.ma_params_frame = ttk.LabelFrame(frame, text="Memetic Algorithm Parameters", padding="10")
        self.ma_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        self._setup_ma_params(self.ma_params_frame)

        self.som_params_frame = ttk.LabelFrame(frame, text="Self-Organizing Map Parameters", padding="10")
        self.som_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        self._setup_som_params(self.som_params_frame)

        self.hybrid_params_frame = ttk.LabelFrame(frame, text="Hybrid GA+SOM Parameters", padding="10") # Added Hybrid params frame
        self.hybrid_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        self._setup_hybrid_params(self.hybrid_params_frame)


        # Run button - span across all algorithm columns + label column
        ttk.Button(frame, text="Run Dimensionality Reduction", command=self._run_dr).grid(row=3, column=0, columnspan=total_colspan, pady=10)

        # Results text area - label in column 0, text area spans the rest
        ttk.Label(frame, text="DR Results:").grid(row=4, column=0, sticky='nw', pady=5, padx=5)
        self.dr_results_text = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=70, height=8)
        self.dr_results_text.grid(row=4, column=1, columnspan=total_algorithm_columns, pady=5, padx=5, sticky='nsew')


        # Configure grid columns to expand (excluding the label column)
        frame.columnconfigure(0, weight=0) # Label column doesn't expand
        for i in range(1, total_colspan): # Columns for radio buttons and results text area
             frame.columnconfigure(i, weight=1)
        frame.rowconfigure(4, weight=1) # Row for the results text area expands

        # Initial toggle to show/hide parameters - defaults to the initial value of self.dr_algorithm_var
        self._toggle_dr_params()


    # --- Parameter Setup Methods ---
    def _setup_pca_params(self, frame):
        """Sets up widgets for PCA parameters."""
        ttk.Label(frame, text="PCA parameters are set by Target Dimension.").grid(row=0, column=0, sticky='w', pady=5, padx=5)

    def _setup_ga_params(self, frame):
         """Sets up widgets for GA parameters."""
         ttk.Label(frame, text="Population Size:").grid(row=0, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ga_pop_size, width=10).grid(row=0, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Generations:").grid(row=1, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ga_n_gen, width=10).grid(row=1, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Mutation Prob:").grid(row=2, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ga_mut_prob, width=10).grid(row=2, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Crossover Prob:").grid(row=3, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ga_cx_prob, width=10).grid(row=3, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Tournament Size:").grid(row=4, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ga_k_tournament, width=10).grid(row=4, column=1, sticky='w', pady=2, padx=5)


    def _setup_de_params(self, frame):
        """Sets up widgets for Differential Evolution parameters."""
        ttk.Label(frame, text="Population Size:").grid(row=0, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.de_pop_size, width=10).grid(row=0, column=1, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="Generations:").grid(row=1, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.de_n_gen, width=10).grid(row=1, column=1, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="Mutation Factor (F):").grid(row=2, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.de_F, width=10).grid(row=2, column=1, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="Crossover Prob (CR):").grid(row=3, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.de_CR, width=10).grid(row=3, column=1, sticky='w', pady=2, padx=5)


    def _setup_ma_params(self, frame):
         """Sets up widgets for Memetic Algorithm parameters."""
         ttk.Label(frame, text="Population Size:").grid(row=0, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ma_pop_size, width=10).grid(row=0, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Generations:").grid(row=1, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ma_n_gen, width=10).grid(row=1, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Mutation Prob:").grid(row=2, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ma_mut_prob, width=10).grid(row=2, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Crossover Prob:").grid(row=3, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ma_cx_prob, width=10).grid(row=3, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Tournament Size:").grid(row=4, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ma_k_tournament, width=10).grid(row=4, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Local Search Prob:").grid(row=5, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.ma_local_search_prob, width=10).grid(row=5, column=1, sticky='w', pady=2, padx=5)


    def _setup_som_params(self, frame):
         """Sets up widgets for SOM parameters."""
         ttk.Label(frame, text="Grid Dim 1 (X):").grid(row=0, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.som_dim1, width=10).grid(row=0, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Grid Dim 2 (Y):").grid(row=1, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.som_dim2, width=10).grid(row=1, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Grid Dim 3 (Z):").grid(row=2, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.som_dim3, width=10).grid(row=2, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Sigma:").grid(row=3, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.som_sigma, width=10).grid(row=3, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Learning Rate:").grid(row=4, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.som_learning_rate, width=10).grid(row=4, column=1, sticky='w', pady=2, padx=5)

         ttk.Label(frame, text="Iterations:").grid(row=5, column=0, sticky='w', pady=2, padx=5)
         ttk.Entry(frame, textvariable=self.som_iterations, width=10).grid(row=5, column=1, sticky='w', pady=2, padx=5)

    def _setup_hybrid_params(self, frame):
        """Sets up widgets for Hybrid GA+SOM parameters."""
        # Intermediate Linear Projection Params
        ttk.Label(frame, text="Intermediate Dim:").grid(row=0, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_intermediate_dim, width=10).grid(row=0, column=1, sticky='w', pady=2, padx=5)

        # SOM Grid Params
        ttk.Label(frame, text="SOM Grid Dim 1 (X):").grid(row=1, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_som_dim1, width=10).grid(row=1, column=1, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="SOM Grid Dim 2 (Y):").grid(row=2, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_som_dim2, width=10).grid(row=2, column=1, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="SOM Grid Dim 3 (Z):").grid(row=3, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_som_dim3, width=10).grid(row=3, column=1, sticky='w', pady=2, padx=5)

        # SOM Training Params
        ttk.Label(frame, text="SOM Sigma:").grid(row=4, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_som_sigma, width=10).grid(row=4, column=1, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="SOM Learning Rate:").grid(row=5, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_som_learning_rate, width=10).grid(row=5, column=1, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="SOM Iterations (Eval):").grid(row=6, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_som_iterations_eval, width=10).grid(row=6, column=1, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="SOM Iterations (Final):").grid(row=7, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_som_iterations_final, width=10).grid(row=7, column=1, sticky='w', pady=2, padx=5)

        # GA Parameters for the Hybrid (using separate variables)
        ttk.Label(frame, text="GA Pop Size:").grid(row=0, column=2, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_ga_pop_size, width=10).grid(row=0, column=3, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="GA Generations:").grid(row=1, column=2, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_ga_n_gen, width=10).grid(row=1, column=3, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="GA Mutation Prob:").grid(row=2, column=2, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_ga_mut_prob, width=10).grid(row=2, column=3, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="GA Crossover Prob:").grid(row=3, column=2, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_ga_cx_prob, width=10).grid(row=3, column=3, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="GA Tournament Size:").grid(row=4, column=2, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.hybrid_ga_k_tournament, width=10).grid(row=4, column=3, sticky='w', pady=2, padx=5)


    def _toggle_dr_params(self):
        """Shows or hides parameter frames based on selected DR algorithm."""
        selected_algorithm = self.dr_algorithm_var.get()

        # Hide all parameter frames
        self.pca_params_frame.grid_forget()
        self.ga_params_frame.grid_forget()
        self.de_params_frame.grid_forget()
        self.ma_params_frame.grid_forget()
        self.som_params_frame.grid_forget()
        self.hybrid_params_frame.grid_forget() # Hide Hybrid frame

        # Determine the correct columnspan based on the number of radio button columns + label column
        # Assuming 6 algorithms, total columns = 7 (label + 6 radio buttons)
        total_colspan = 7 # Update if you add more algorithms/radio buttons

        # Show the selected algorithm's parameter frame, ensuring correct columnspan
        if selected_algorithm == "PCA":
            self.pca_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        elif selected_algorithm == "Genetic Algorithm":
            self.ga_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        elif selected_algorithm == "Differential Evolution":
             self.de_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        elif selected_algorithm == "Memetic Algorithm":
             self.ma_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        elif selected_algorithm == "Self-Organizing Map":
             self.som_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)
        elif selected_algorithm == "Hybrid GA+SOM": # Show Hybrid frame
             self.hybrid_params_frame.grid(row=2, column=0, columnspan=total_colspan, sticky='ew', pady=10, padx=5)


    def _run_dr(self):
        """Runs the selected dimensionality reduction algorithm."""
        if self.X_train_scaled is None or self.X_test_scaled is None or self.y_train is None or self.y_test is None:
            messagebox.showwarning("DR Error", "Please load and preprocess data first.")
            return

        selected_algorithm = self.dr_algorithm_var.get()
        target_dim = self.target_dim.get()

        # Input validation based on selected algorithm
        n_features = self.X_train_scaled.shape[1] if self.X_train_scaled is not None else 0 # Handle case before preprocessing

        # General target_dim validation for methods producing a fixed dimension output
        if selected_algorithm in ["PCA", "Genetic Algorithm", "Differential Evolution", "Memetic Algorithm"]:
             if target_dim <= 0 or target_dim > n_features:
                  messagebox.showwarning("DR Error", f"Invalid target dimension ({target_dim}) for {selected_algorithm}. Must be between 1 and number of features ({n_features}).")
                  return
        elif selected_algorithm in ["Self-Organizing Map", "Hybrid GA+SOM"]:
             # SOM and Hybrid output dimension is determined by grid dimensions, but target_dim can guide the user
             # We still need target_dim to be >= 1 as a general requirement for DR
             if target_dim <= 0:
                 messagebox.showwarning("DR Error", f"Invalid target dimension ({target_dim}). Must be at least 1.")
                 return

             # Specific SOM/Hybrid grid dimension validation
             if selected_algorithm == "Self-Organizing Map":
                  grid_dims_check = []
                  if target_dim >= 1: grid_dims_check.append(self.som_dim1.get())
                  if target_dim >= 2: grid_dims_check.append(self.som_dim2.get())
                  if target_dim >= 3: grid_dims_check.append(self.som_dim3.get())

                  if any(dim <= 0 for dim in grid_dims_check) or len(grid_dims_check) < target_dim:
                       messagebox.showwarning("SOM Error", "SOM grid dimensions must be greater than 0 and match the target dimension count.")
                       return
                  total_som_neurons = np.prod(grid_dims_check)
                  if total_som_neurons <= 0:
                       messagebox.showwarning("SOM Error", "Total number of SOM neurons must be greater than 0.")
                       return

             elif selected_algorithm == "Hybrid GA+SOM":
                  intermediate_dim = self.hybrid_intermediate_dim.get()
                  if intermediate_dim <= 0 or intermediate_dim > n_features:
                       messagebox.showwarning("Hybrid GA+SOM Error", f"Invalid intermediate dimension ({intermediate_dim}). Must be between 1 and number of features ({n_features}).")
                       return

                  # Check SOM grid dims for Hybrid (based on main target_dim)
                  hybrid_som_grid_dims_list = []
                  if target_dim >= 1: hybrid_som_grid_dims_list.append(self.hybrid_som_dim1.get())
                  if target_dim >= 2: hybrid_som_grid_dims_list.append(self.hybrid_som_dim2.get())
                  if target_dim >= 3: hybrid_som_grid_dims_list.append(self.hybrid_som_dim3.get())


                  if any(dim <= 0 for dim in hybrid_som_grid_dims_list) or len(hybrid_som_grid_dims_list) < target_dim:
                       messagebox.showwarning("Hybrid GA+SOM Error", "Hybrid SOM grid dimensions must be greater than 0 and match the target dimension count.")
                       return
                  total_hybrid_som_neurons = np.prod(hybrid_som_grid_dims_list)
                  if total_hybrid_som_neurons <= 0:
                       messagebox.showwarning("Hybrid GA+SOM Error", "Total number of Hybrid SOM neurons must be greater than 0.")
                       return


        self.dr_results_text.delete('1.0', tk.END)
        self.dr_results_text.insert(tk.END, f"Running {selected_algorithm}...\n")
        self._reset_dr_results() # Reset previous DR results

        try:
            # Use Logistic Regression as the default evaluation model for fitness for heuristic algorithms
            # Pass a new instance each time to ensure it's not already fitted
            eval_model_for_fitness = LogisticRegression(max_iter=self.lr_max_iter.get(), solver=self.lr_solver.get(), random_state=self.random_state.get())


            if selected_algorithm == "PCA":
                self.pca_dr_model = PCADR(n_components=target_dim, random_state=self.random_state.get())
                self.pca_dr_model.fit(self.X_train_scaled)
                if self.X_all_scaled is not None:
                     self.X_pca_viz = self.pca_dr_model.transform(self.X_all_scaled)
                # Evaluate PCA using the selected evaluation model
                evaluator = ModelEvaluator(model=self._get_selected_evaluation_model())
                X_test_pca = self.pca_dr_model.transform(self.X_test_scaled)
                self.pca_accuracy = evaluator.evaluate(self.pca_dr_model.transform(self.X_train_scaled), X_test_pca, self.y_train, self.y_test)

                self.dr_results_text.insert(tk.END, "\nPCA Completed.\n")
                if self.pca_dr_model.pca is not None:
                     self.dr_results_text.insert(tk.END, f"Explained Variance Ratio: {self.pca_dr_model.pca.explained_variance_ratio_.sum():.4f}\n")
                if self.X_pca_viz is not None:
                     self.dr_results_text.insert(tk.END, f"Projected Data Shape: {self.X_pca_viz.shape}\n")
                if self.pca_accuracy is not None:
                     self.dr_results_text.insert(tk.END, f"Accuracy on Test Set Projection (using selected eval model): {self.pca_accuracy:.4f}\n")


            elif selected_algorithm == "Genetic Algorithm":
                if self.X_train_scaled is None: raise ValueError("Training data not available for GA.")
                self.ga_dr_model = GeneticAlgorithmDR(
                    n_features=self.X_train_scaled.shape[1],
                    target_dim=target_dim,
                    pop_size=self.ga_pop_size.get(),
                    n_gen=self.ga_n_gen.get(),
                    mut_prob=self.ga_mut_prob.get(),
                    cx_prob=self.ga_cx_prob.get(),
                    k_tournament=self.ga_k_tournament.get(),
                    evaluation_model=eval_model_for_fitness # Use LR for GA fitness
                )
                self.ga_dr_model.fit(self.X_train_scaled, self.X_test_scaled, self.y_train, self.y_test)
                if self.X_all_scaled is not None:
                     self.Z_best_ga_viz = self.ga_dr_model.transform(self.X_all_scaled)
                # Evaluate GA using the selected evaluation model
                evaluator = ModelEvaluator(model=self._get_selected_evaluation_model())
                Z_test_ga = self.ga_dr_model.transform(self.X_test_scaled)
                self.ga_accuracy = evaluator.evaluate(self.ga_dr_model.transform(self.X_train_scaled), Z_test_ga, self.y_train, self.y_test)

                self.dr_results_text.insert(tk.END, "\nGenetic Algorithm Completed.\n")
                if self.Z_best_ga_viz is not None:
                     self.dr_results_text.insert(tk.END, f"Projected Data Shape: {self.Z_best_ga_viz.shape}\n")
                if self.ga_accuracy is not None:
                     self.dr_results_text.insert(tk.END, f"Accuracy on Test Set Projection (using selected eval model): {self.ga_accuracy:.4f}\n")


            elif selected_algorithm == "Differential Evolution":
                 if self.X_train_scaled is None: raise ValueError("Training data not available for DE.")
                 self.de_dr_model = DifferentialEvolutionDR(
                     n_features=self.X_train_scaled.shape[1],
                     target_dim=target_dim,
                     pop_size=self.de_pop_size.get(),
                     n_gen=self.de_n_gen.get(),
                     F=self.de_F.get(),
                     CR=self.de_CR.get(),
                     evaluation_model=eval_model_for_fitness, # Use LR for DE fitness
                     random_state=self.random_state.get()
                 )
                 self.de_dr_model.fit(self.X_train_scaled, self.X_test_scaled, self.y_train, self.y_test)
                 if self.X_all_scaled is not None:
                     self.Z_best_de_viz = self.de_dr_model.transform(self.X_all_scaled)
                 # Evaluate DE using the selected evaluation model
                 evaluator = ModelEvaluator(model=self._get_selected_evaluation_model())
                 Z_test_de = self.de_dr_model.transform(self.X_test_scaled)
                 self.de_accuracy = evaluator.evaluate(self.de_dr_model.transform(self.X_train_scaled), Z_test_de, self.y_train, self.y_test)

                 self.dr_results_text.insert(tk.END, "\nDifferential Evolution Completed.\n")
                 if self.Z_best_de_viz is not None:
                     self.dr_results_text.insert(tk.END, f"Projected Data Shape: {self.Z_best_de_viz.shape}\n")
                 if self.de_accuracy is not None:
                     self.dr_results_text.insert(tk.END, f"Accuracy on Test Set Projection (using selected eval model): {self.de_accuracy:.4f}\n")


            elif selected_algorithm == "Memetic Algorithm":
                 if self.X_train_scaled is None: raise ValueError("Training data not available for MA.")
                 self.ma_dr_model = MemeticAlgorithmDR(
                     n_features=self.X_train_scaled.shape[1],
                     target_dim=target_dim,
                     pop_size=self.ma_pop_size.get(),
                     n_gen=self.ma_n_gen.get(),
                     mut_prob=self.ma_mut_prob.get(),
                     cx_prob=self.ma_cx_prob.get(),
                     k_tournament=self.ma_k_tournament.get(),
                     local_search_prob=self.ma_local_search_prob.get(),
                     evaluation_model=eval_model_for_fitness, # Use LR for MA fitness
                     random_state=self.random_state.get()
                 )
                 self.ma_dr_model.fit(self.X_train_scaled, self.X_test_scaled, self.y_train, self.y_test)
                 if self.X_all_scaled is not None:
                      self.Z_best_ma_viz = self.ma_dr_model.transform(self.X_all_scaled)
                 # Evaluate MA using the selected evaluation model
                 evaluator = ModelEvaluator(model=self._get_selected_evaluation_model())
                 Z_test_ma = self.ma_dr_model.transform(self.X_test_scaled)
                 self.ma_accuracy = evaluator.evaluate(self.ma_dr_model.transform(self.X_train_scaled), Z_test_ma, self.y_train, self.y_test)

                 self.dr_results_text.insert(tk.END, "\nMemetic Algorithm Completed.\n")
                 if self.Z_best_ma_viz is not None:
                      self.dr_results_text.insert(tk.END, f"Projected Data Shape: {self.Z_best_ma_viz.shape}\n")
                 if self.ma_accuracy is not None:
                      self.dr_results_text.insert(tk.END, f"Accuracy on Test Set Projection (using selected eval model): {self.ma_accuracy:.4f}\n")


            elif selected_algorithm == "Self-Organizing Map":
                 if self.X_train_scaled is None: raise ValueError("Training data not available for SOM.")

                 # Determine SOM grid dimensions based on target_dim from the main DR settings
                 # The SOM output dimension will be the product of these grid dimensions
                 som_grid_dims_list = []
                 if target_dim >= 1: som_grid_dims_list.append(self.som_dim1.get())
                 if target_dim >= 2: som_grid_dims_list.append(self.som_dim2.get())
                 if target_dim >= 3: som_grid_dims_list.append(self.som_dim3.get())

                 # Re-validate grid dimensions after getting them
                 if any(dim <= 0 for dim in som_grid_dims_list) or len(som_grid_dims_list) < target_dim:
                      messagebox.showwarning("SOM Error", "SOM grid dimensions must be greater than 0 and match the target dimension count.")
                      return

                 # If target_dim > 3, decide how to handle (warn, error, use only first 3 dims)
                 if target_dim > 3:
                      messagebox.showwarning("SOM Warning", f"SOM visualization is only supported for 3D or less. Using first 3 specified SOM grid dimensions for visualization if available.")


                 self.som_dr_model = SOMDR(
                      grid_dims=tuple(som_grid_dims_list), # Pass the tuple of grid dims
                      sigma=self.som_sigma.get(),
                      learning_rate=self.som_learning_rate.get(),
                      num_iterations=self.som_iterations.get(),
                      random_state=self.random_state.get()
                 )
                 # SOM fit only needs X_train
                 self.som_dr_model.fit(self.X_train_scaled)

                 # Transform the full scaled dataset for visualization and test set for evaluation
                 if self.X_all_scaled is not None:
                      self.Z_som_viz = self.som_dr_model.transform(self.X_all_scaled)
                 Z_som_test = self.som_dr_model.transform(self.X_test_scaled)

                 # Evaluate SOM using the selected evaluation model
                 evaluator = ModelEvaluator(model=self._get_selected_evaluation_model())
                 # Only evaluate if transformation was successful and test data is available
                 if Z_som_test is not None:
                      # Pass the projected test data shape to evaluation to ensure compatibility if needed
                      self.som_accuracy = evaluator.evaluate(self.som_dr_model.transform(self.X_train_scaled), Z_som_test, self.y_train, self.y_test)
                 else:
                      self.som_accuracy = None

                 self.dr_results_text.insert(tk.END, "\nSelf-Organizing Map Completed.\n")
                 if self.Z_som_viz is not None:
                      self.dr_results_text.insert(tk.END, f"Projected Data Shape: {self.Z_som_viz.shape}\n")
                 if self.som_accuracy is not None:
                      self.dr_results_text.insert(tk.END, f"Accuracy on Test Set Projection (using selected eval model): {self.som_accuracy:.4f}\n")
                 else:
                      self.dr_results_text.insert(tk.END, "Accuracy on Test Set Projection: Could not evaluate (check SOM parameters and target dimension).\n")


            elif selected_algorithm == "Hybrid GA+SOM": # Added Hybrid GA+SOM logic
                 if self.X_train_scaled is None: raise ValueError("Training data not available for Hybrid.")

                 # Parameters for Hybrid
                 intermediate_dim = self.hybrid_intermediate_dim.get()
                 hybrid_som_grid_dims_list = []
                 target_dim_final = self.target_dim.get() # Final output dim from main setting
                 if target_dim_final >= 1: hybrid_som_grid_dims_list.append(self.hybrid_som_dim1.get())
                 if target_dim_final >= 2: hybrid_som_grid_dims_list.append(self.hybrid_som_dim2.get())
                 if target_dim_final >= 3: hybrid_som_grid_dims_list.append(self.hybrid_som_dim3.get())


                 # Re-validate grid dimensions for Hybrid
                 if any(dim <= 0 for dim in hybrid_som_grid_dims_list) or len(hybrid_som_grid_dims_list) < target_dim_final:
                      messagebox.showwarning("Hybrid GA+SOM Error", "Hybrid SOM grid dimensions must be greater than 0 and match the target dimension count.")
                      return

                 if target_dim_final > 3:
                      messagebox.showwarning("Hybrid GA+SOM Warning", f"Hybrid GA+SOM visualization is only supported for 3D or less. Using first 3 specified SOM grid dimensions for visualization if available.")


                 self.hybrid_dr_model = HybridGASOMDR(
                      n_features=self.X_train_scaled.shape[1],
                      intermediate_dim=intermediate_dim,
                      som_grid_dims=tuple(hybrid_som_grid_dims_list), # Pass the tuple of grid dims
                      ga_pop_size=self.hybrid_ga_pop_size.get(), # Use hybrid specific GA params
                      ga_n_gen=self.hybrid_ga_n_gen.get(),
                      ga_mut_prob=self.hybrid_ga_mut_prob.get(),
                      ga_cx_prob=self.hybrid_ga_cx_prob.get(),
                      ga_k_tournament=self.hybrid_ga_k_tournament.get(),
                      som_sigma=self.hybrid_som_sigma.get(),
                      som_learning_rate=self.hybrid_som_learning_rate.get(),
                      som_iterations_eval=self.hybrid_som_iterations_eval.get(),
                      som_iterations_final=self.hybrid_som_iterations_final.get(),
                      evaluation_model_fitness=eval_model_for_fitness, # Use LR for GA fitness
                      random_state=self.random_state.get()
                 )
                 # Fit the hybrid model
                 self.hybrid_dr_model.fit(self.X_train_scaled, self.X_test_scaled, self.y_train, self.y_test)

                 # Transform the full scaled dataset for visualization
                 if self.X_all_scaled is not None:
                      self.Z_best_hybrid_viz = self.hybrid_dr_model.transform(self.X_all_scaled)

                 # Evaluate the hybrid projection using the selected evaluation model
                 evaluator = ModelEvaluator(model=self._get_selected_evaluation_model())
                 Z_hybrid_test = self.hybrid_dr_model.transform(self.X_test_scaled)

                 if Z_hybrid_test is not None:
                      self.hybrid_accuracy = evaluator.evaluate(self.hybrid_dr_model.transform(self.X_train_scaled), Z_hybrid_test, self.y_train, self.y_test)
                 else:
                      self.hybrid_accuracy = None


                 self.dr_results_text.insert(tk.END, "\nHybrid GA+SOM Completed.\n")
                 if self.Z_best_hybrid_viz is not None:
                      self.dr_results_text.insert(tk.END, f"Projected Data Shape: {self.Z_best_hybrid_viz.shape}\n")
                 if self.hybrid_accuracy is not None:
                      self.dr_results_text.insert(tk.END, f"Accuracy on Test Set Projection (using selected eval model): {self.hybrid_accuracy:.4f}\n")
                 else:
                      self.dr_results_text.insert(tk.END, "Accuracy on Test Set Projection: Could not evaluate.\n")


            messagebox.showinfo("DR Success", f"{selected_algorithm} completed successfully!")

        except ValueError as e:
             messagebox.showerror("DR Error", f"Input error during DR: {e}")
             self.dr_results_text.insert(tk.END, f"\nError: {e}")
             self._reset_dr_results()
        except Exception as e:
            messagebox.showerror("DR Error", f"An error occurred during {selected_algorithm} DR: {e}")
            self.dr_results_text.insert(tk.END, f"\nError: {e}")
            self._reset_dr_results()


    def _reset_dr_results(self):
         """Resets DR results and subsequent step results."""
         self.ga_dr_model = None
         self.pca_dr_model = None
         self.de_dr_model = None
         self.ma_dr_model = None
         self.som_dr_model = None
         self.hybrid_dr_model = None

         self.Z_best_ga_viz = None
         self.X_pca_viz = None
         self.Z_best_de_viz = None
         self.Z_best_ma_viz = None
         self.Z_som_viz = None
         self.Z_best_hybrid_viz = None

         self.ga_accuracy = None
         self.pca_accuracy = None
         self.de_accuracy = None
         self.ma_accuracy = None
         self.som_accuracy = None
         self.hybrid_accuracy = None

         self.dr_results_text.delete('1.0', tk.END)
         self.dr_results_text.insert(tk.END, "DR Results: Results reset or failed.")


    # --- Evaluation Tab Methods ---
    def _setup_evaluation_tab(self, frame):
        """Sets up widgets for the Evaluation tab."""
        ttk.Label(frame, text="Select Model:").grid(row=0, column=0, sticky='w', pady=5, padx=5)
        # Populate with available evaluation models
        self.eval_model_combobox = ttk.Combobox(frame, textvariable=self.eval_model_name, values=["Logistic Regression", "Random Forest"], state="readonly")
        self.eval_model_combobox.grid(row=0, column=1, sticky='w', pady=5, padx=5)
        self.eval_model_combobox.bind('<<ComboboxSelected>>', self._update_eval_params_display)

        # Parameter frames for evaluation models
        self.lr_params_frame = ttk.LabelFrame(frame, text="Logistic Regression Parameters", padding="10")
        self.lr_params_frame.grid(row=1, column=0, columnspan=2, sticky='ew', pady=10, padx=5)
        self._setup_lr_params(self.lr_params_frame)

        self.rf_params_frame = ttk.LabelFrame(frame, text="Random Forest Parameters", padding="10")
        self.rf_params_frame.grid(row=1, column=0, columnspan=2, sticky='ew', pady=10, padx=5)
        self._setup_rf_params(self.rf_params_frame)

        ttk.Button(frame, text="Evaluate DR Result", command=self._run_evaluation).grid(row=2, column=0, columnspan=2, pady=10)

        ttk.Label(frame, text="Evaluation Results:").grid(row=3, column=0, sticky='nw', pady=5, padx=5)
        self.evaluation_results_text = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=70, height=8)
        self.evaluation_results_text.grid(row=3, column=1, columnspan=2, pady=5, padx=5, sticky='nsew')

        frame.columnconfigure(1, weight=1)
        frame.rowconfigure(3, weight=1)

        # Initial display of evaluation parameters
        self._update_eval_params_display()

    def _setup_lr_params(self, frame):
        """Sets up widgets for Logistic Regression parameters."""
        ttk.Label(frame, text="Max Iterations:").grid(row=0, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.lr_max_iter, width=10).grid(row=0, column=1, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="Solver:").grid(row=1, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.lr_solver, width=10).grid(row=1, column=1, sticky='w', pady=2, padx=5)

    def _setup_rf_params(self, frame):
        """Sets up widgets for Random Forest parameters (Placeholder)."""
        ttk.Label(frame, text="Random Forest parameters").grid(row=0, column=0, sticky='w', pady=5, padx=5)
        ttk.Label(frame, text="Number of Estimators:").grid(row=1, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.rf_n_estimators, width=10).grid(row=1, column=1, sticky='w', pady=2, padx=5)
        ttk.Label(frame, text="Max Depth:").grid(row=2, column=0, sticky='w', pady=2, padx=5)
        ttk.Entry(frame, textvariable=self.rf_max_depth, width=10).grid(row=2, column=1, sticky='w', pady=2, padx=5)
        
        # Add more parameter inputs here

    def _update_eval_params_display(self, event=None):
        """Shows or hides evaluation model parameter frames."""
        selected_model = self.eval_model_name.get()
        self.lr_params_frame.grid_forget()
        self.rf_params_frame.grid_forget()

        if selected_model == "Logistic Regression":
            self.lr_params_frame.grid(row=1, column=0, columnspan=2, sticky='ew', pady=10, padx=5)
        elif selected_model == "Random Forest":
            self.rf_params_frame.grid(row=1, column=0, columnspan=2, sticky='ew', pady=10, padx=5)


    def _get_selected_evaluation_model(self):
        """Instantiates the selected evaluation model with parameters from GUI."""
        selected_model_name = self.eval_model_name.get()
        random_state = self.random_state.get()

        try:
            if selected_model_name == "Logistic Regression":
                return LogisticRegression(
                    max_iter=self.lr_max_iter.get(),
                    solver=self.lr_solver.get(),
                    random_state=random_state
                )
            elif selected_model_name == "Random Forest":
                 
                 return RandomForestClassifier(
                      n_estimators=self.rf_n_estimators.get(),
                      max_depth=self.rf_max_depth.get(),
                      random_state=random_state
                      )# Return None for placeholder models
            else:
                messagebox.showwarning("Unknown Model", f"Unknown evaluation model: {selected_model_name}")
                return None
        except Exception as e:
             messagebox.showerror("Model Instantiation Error", f"Could not create evaluation model: {e}")
             return None


    def _run_evaluation(self):
        """Runs evaluation on the currently selected DR result using the selected evaluation model."""
        if self.X_train_scaled is None or self.y_train is None or self.X_test_scaled is None or self.y_test is None:
            messagebox.showwarning("Evaluation Error", "Please load and preprocess data first.")
            return

        # Check if any DR method has been run and has results
        if all([m is None for m in [self.ga_dr_model, self.pca_dr_model, self.de_dr_model, self.ma_dr_model, self.som_dr_model, self.hybrid_dr_model]]):
             messagebox.showwarning("Evaluation Error", "Please run Dimensionality Reduction first.")
             return

        # Determine which DR result to evaluate based on the currently selected DR algorithm in the DR tab
        selected_dr_method = self.dr_algorithm_var.get()
        X_train_proj = None
        X_test_proj = None
        dr_method_evaluated = "None"

        if selected_dr_method == "Genetic Algorithm" and self.ga_dr_model is not None:
             X_train_proj = self.ga_dr_model.transform(self.X_train_scaled)
             X_test_proj = self.ga_dr_model.transform(self.X_test_scaled)
             dr_method_evaluated = "Genetic Algorithm"
        elif selected_dr_method == "PCA" and self.pca_dr_model is not None:
             X_train_proj = self.pca_dr_model.transform(self.X_train_scaled)
             X_test_proj = self.pca_dr_model.transform(self.X_test_scaled)
             dr_method_evaluated = "PCA"
        elif selected_dr_method == "Differential Evolution" and self.de_dr_model is not None:
             X_train_proj = self.de_dr_model.transform(self.X_train_scaled)
             X_test_proj = self.de_dr_model.transform(self.X_test_scaled)
             dr_method_evaluated = "Differential Evolution"
        elif selected_dr_method == "Memetic Algorithm" and self.ma_dr_model is not None:
             X_train_proj = self.ma_dr_model.transform(self.X_train_scaled)
             X_test_proj = self.ma_dr_model.transform(self.X_test_scaled)
             dr_method_evaluated = "Memetic Algorithm"
        elif selected_dr_method == "Self-Organizing Map" and self.som_dr_model is not None:
             X_train_proj = self.som_dr_model.transform(self.X_train_scaled)
             X_test_proj = self.som_dr_model.transform(self.X_test_scaled)
             dr_method_evaluated = "Self-Organizing Map"
        elif selected_dr_method == "Hybrid GA+SOM" and self.hybrid_dr_model is not None:
             X_train_proj = self.hybrid_dr_model.transform(self.X_train_scaled)
             X_test_proj = self.hybrid_dr_model.transform(self.X_test_scaled)
             dr_method_evaluated = "Hybrid GA+SOM"


        if X_train_proj is None or X_test_proj is None:
             messagebox.showwarning("Evaluation Error", f"No {dr_method_evaluated} results available for evaluation or transformation failed.")
             return

        # Check if projected data is empty or has zero dimensions which would cause evaluation errors
        if X_train_proj.size == 0 or X_test_proj.size == 0 or X_train_proj.shape[1] == 0 or X_test_proj.shape[1] == 0:
             messagebox.showwarning("Evaluation Error", f"Projected data for {dr_method_evaluated} is empty or has zero dimensions. Cannot evaluate.")
             self.evaluation_results_text.delete('1.0', tk.END)
             self.evaluation_results_text.insert(tk.END, f"Evaluation Failed: Projected data is invalid.")
             return


        eval_model = self._get_selected_evaluation_model()
        if eval_model is None:
             # Error message handled in _get_selected_evaluation_model
             return

        self.evaluation_results_text.delete('1.0', tk.END)
        self.evaluation_results_text.insert(tk.END, f"Evaluating {dr_method_evaluated} projection using {self.eval_model_name.get()}...\n")

        try:
            evaluator = ModelEvaluator(model=eval_model)
            # The ModelEvaluator.evaluate method already fits and predicts
            accuracy = evaluator.evaluate(X_train_proj, X_test_proj, self.y_train, self.y_test)

            self.evaluation_results_text.insert(tk.END, "\nEvaluation Completed.\n")
            self.evaluation_results_text.insert(tk.END, f"Accuracy on Test Set ({dr_method_evaluated} projection): {accuracy:.4f}\n")

            # Store the accuracy result in the corresponding variable
            if dr_method_evaluated == "Genetic Algorithm":
                 self.ga_accuracy = accuracy
            elif dr_method_evaluated == "PCA":
                 self.pca_accuracy = accuracy
            elif dr_method_evaluated == "Differential Evolution":
                 self.de_accuracy = accuracy
            elif dr_method_evaluated == "Memetic Algorithm":
                 self.ma_accuracy = accuracy
            elif dr_method_evaluated == "Self-Organizing Map":
                 self.som_accuracy = accuracy
            elif dr_method_evaluated == "Hybrid GA+SOM":
                 self.hybrid_accuracy = accuracy


            messagebox.showinfo("Evaluation Success", f"Evaluation completed successfully! Accuracy: {accuracy:.4f}")

        except Exception as e:
            messagebox.showerror("Evaluation Error", f"An error occurred during evaluation: {e}")
            self.evaluation_results_text.insert(tk.END, f"\nEvaluation Failed: {e}")


    # --- Visualization Tab Methods ---
    def _setup_visualization_tab(self, frame):
        """Sets up widgets for the Visualization tab."""
        ttk.Label(frame, text="Select DR Result to Visualize:").grid(row=0, column=0, sticky='w', pady=5, padx=5)
        self.viz_dr_result_var = tk.StringVar(value="Genetic Algorithm") # Default selection

        # Add radio buttons for selecting which DR result to visualize
        # Adjust column numbers based on total number of algorithms (6 in this case)
        ttk.Radiobutton(frame, text="PCA", variable=self.viz_dr_result_var, value="PCA").grid(row=0, column=1, sticky='w', pady=5, padx=5)
        ttk.Radiobutton(frame, text="Genetic Algorithm", variable=self.viz_dr_result_var, value="Genetic Algorithm").grid(row=0, column=2, sticky='w', pady=5, padx=5)
        ttk.Radiobutton(frame, text="Differential Evolution", variable=self.viz_dr_result_var, value="Differential Evolution").grid(row=0, column=3, sticky='w', pady=5, padx=5)
        ttk.Radiobutton(frame, text="Memetic Algorithm", variable=self.viz_dr_result_var, value="Memetic Algorithm").grid(row=0, column=4, sticky='w', pady=5, padx=5)
        ttk.Radiobutton(frame, text="Self-Organizing Map", variable=self.viz_dr_result_var, value="Self-Organizing Map").grid(row=0, column=5, sticky='w', pady=5, padx=5)
        ttk.Radiobutton(frame, text="Hybrid GA+SOM", variable=self.viz_dr_result_var, value="Hybrid GA+SOM").grid(row=0, column=6, sticky='w', pady=5, padx=5) # Added Hybrid


        # Generate Plot button - span across all algorithm columns + label column (total 7 columns)
        ttk.Button(frame, text="Generate Plot", command=self._generate_plot).grid(row=1, column=0, columnspan=7, pady=10)

        # Frame to embed the matplotlib plot - span across all algorithm columns + label column
        self.plot_frame = ttk.Frame(frame)
        self.plot_frame.grid(row=2, column=0, columnspan=7, pady=10, padx=5, sticky='nsew')

        # Configure grid for resizing - add weight for new columns
        frame.columnconfigure(0, weight=0) # Label column doesn't expand
        for i in range(1, 7): # Columns for radio buttons and plot frame
             frame.columnconfigure(i, weight=1)
        frame.rowconfigure(2, weight=1) # Row for the plot frame expands


    def _generate_plot(self):
        """Generates and displays the 3D plot for the selected DR result."""
        selected_result = self.viz_dr_result_var.get()
        X_to_plot = None
        accuracy_to_show = None
        title_suffix = ""
        cmap_to_use = 'viridis' # Default colormap

        # Get the correct data and accuracy based on selected algorithm
        if selected_result == "PCA":
             X_to_plot = self.X_pca_viz
             accuracy_to_show = self.pca_accuracy
             # Use the target_dim from the GUI for the title suffix
             title_suffix = f"PCA Projection ({self.target_dim.get()}D)"
             cmap_to_use = 'viridis'
        elif selected_result == "Genetic Algorithm":
             X_to_plot = self.Z_best_ga_viz
             accuracy_to_show = self.ga_accuracy
             # Use the target_dim from the GUI for the title suffix
             title_suffix = f"GA Projection ({self.target_dim.get()}D)"
             cmap_to_use = 'coolwarm' # Example colormap
        elif selected_result == "Differential Evolution":
             X_to_plot = self.Z_best_de_viz
             accuracy_to_show = self.de_accuracy
             # Use the target_dim from the GUI for the title suffix
             title_suffix = f"DE Projection ({self.target_dim.get()}D)"
             cmap_to_use = 'plasma' # Example colormap
        elif selected_result == "Memetic Algorithm":
             X_to_plot = self.Z_best_ma_viz
             accuracy_to_show = self.ma_accuracy
             # Use the target_dim from the GUI for the title suffix
             title_suffix = f"MA Projection ({self.target_dim.get()}D)"
             cmap_to_use = 'cividis' # Example colormap
        elif selected_result == "Self-Organizing Map":
             X_to_plot = self.Z_som_viz
             accuracy_to_show = self.som_accuracy
             # For SOM, the output dimension is determined by the grid_dims, not necessarily target_dim
             # Check the actual shape of the projected data if available
             if X_to_plot is not None:
                  title_suffix = f"SOM Projection ({X_to_plot.shape[1]}D)"
             else:
                  # Fallback if data isn't ready, use the target_dim which drove the SOM init
                  title_suffix = f"SOM Projection (Target {self.target_dim.get()}D)"

             cmap_to_use = 'Spectral' # Example colormap
        elif selected_result == "Hybrid GA+SOM":
             X_to_plot = self.Z_best_hybrid_viz
             accuracy_to_show = self.hybrid_accuracy
             # Use the actual output dimension if available from transformed data
             if X_to_plot is not None:
                  title_suffix = f"Hybrid GA+SOM Projection ({X_to_plot.shape[1]}D)"
             else:
                   # Fallback if data isn't ready, use the target_dim which drove the SOM init
                  title_suffix = f"Hybrid GA+SOM Projection (Target {self.target_dim.get()}D)"
             cmap_to_use = 'Set1' # Example colormap from your snippet


        if X_to_plot is None or self.y_all_viz is None:
             messagebox.showwarning("Visualization Error", f"No {selected_result} results available for plotting. Please run DR first.")
             return

        # Check if the data is 3-dimensional for 3D plotting
        n_dims = X_to_plot.shape[1]
        if n_dims not in [2, 3]:
          messagebox.showwarning("Visualization Error", f"Visualization requires 2 or 3 dimensions, but selected data has {n_dims}.")
          return

        try:
             # Clear previous plot
             for widget in self.plot_frame.winfo_children():
                  widget.destroy()
             fig = plt.Figure(figsize=(6, 5), dpi=100)
             if n_dims == 2:
               ax = fig.add_subplot(111)
               scatter = ax.scatter(
                    X_to_plot[:, 0],
                    X_to_plot[:, 1],
                    c=self.y_all_viz,
                    cmap=cmap_to_use,
                    alpha=0.7
               )
               ax.set_xlabel(f'{selected_result} Component 1')
               ax.set_ylabel(f'{selected_result} Component 2')
               ax.grid(True)
             else:  # 3D plot
               ax = fig.add_subplot(111, projection='3d')
               # Create the 3D scatter plot
               scatter = ax.scatter(
                    X_to_plot[:, 0],
                    X_to_plot[:, 1],
                    X_to_plot[:, 2],
                    c=self.y_all_viz, # Color points by the original class labels
                    cmap=cmap_to_use,
                    alpha=0.7
               )

             # Set title and labels, including accuracy if available
               if accuracy_to_show is not None:
                    ax.set_title(f'{title_suffix} (Acc: {accuracy_to_show:.3f})')
               else:
                    ax.set_title(title_suffix)

               ax.set_xlabel(f'{selected_result} Component 1')
               ax.set_ylabel(f'{selected_result} Component 2')
               ax.set_zlabel(f'{selected_result} Component 3')

             # Add legend for classes
             legend = ax.legend(*scatter.legend_elements(), title="Classes")
             ax.add_artist(legend)

             # Embed the plot in the Tkinter window
             canvas = FigureCanvasTkAgg(fig, master=self.plot_frame)
             canvas.draw()
             canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        except Exception as e:
             messagebox.showerror("Visualization Error", f"An error occurred during plotting: {e}")


# --- Main Execution Block ---
if __name__ == "__main__":
    # This block runs when the script is executed directly
    # Check if essential libraries are available
    try:
        import pandas as pd
        import numpy as np
        import sklearn
        import matplotlib
        import minisom # Check for minisom specifically as it's used by SOM/Hybrid
    except ImportError as e:
        messagebox.showerror("Library Error", f"Required library not found: {e}.\nPlease install it using pip (e.g., 'pip install pandas numpy scikit-learn matplotlib minisom').")
        exit() # Exit if essential libraries are missing

    # Determine the project root to check for the src directory
    # This block includes robust path handling for notebook environments
    project_root = None
    try:
        # Attempt to get the script directory if __file__ is defined
        # Assuming the script is within the project structure (like in 'notebooks' or 'src')
        script_dir = os.path.dirname(os.path.abspath(__file__))
        # Project root is parent of script directory
        project_root = os.path.abspath(os.path.join(script_dir, os.pardir))

    except NameError:
        # If __file__ is not defined, use the current working directory
        # Assume current working directory is within the project structure (like 'notebooks')
        current_working_dir = os.getcwd()
        # Project root is parent of current working directory
        project_root = os.path.abspath(os.path.join(current_working_dir, os.pardir))
        # Optional: Add a message box for debugging - requires root window later
        # messagebox.showwarning("Environment Warning", f"Could not determine script directory using __file__. Assuming project root based on CWD: '{project_root}'.")


    # Now construct the path to the src directory relative to the determined project root
    src_dir = os.path.join(project_root, 'src')

    # Check if src directory and necessary files exist
    # List all required files from the imports to ensure they are present
    required_src_files_from_imports = ['data_loader.py', 'preprocessing.py', 'dimensionality_reduction.py', 'evaluation.py'] # Add visualization.py if separate

    missing_files_check = [f for f in required_src_files_from_imports if not os.path.exists(os.path.join(src_dir, f))]


    if not os.path.exists(src_dir) or missing_files_check:
        error_message = f"Missing 'src' directory or essential files.\nExpected src directory at: {src_dir}\nPlease ensure you have a 'src' directory in your project root ({project_root}), containing:\n{', '.join(required_src_files_from_imports)}"
        if missing_files_check:
            error_message += f"\n\nMissing specific files within src: {', '.join(missing_files_check)}"
        # We cannot use messagebox here before root is created, print to console instead
        print("Project Structure Error:")
        print(error_message)
        # Use a simple dialog if running as script, or rely on notebook output
        try:
             tk_root = tk.Tk()
             tk_root.withdraw() # Hide the main window
             messagebox.showerror("Project Structure Error", error_message)
             tk_root.destroy()
        except:
             pass # If Tkinter is not available, just print

        exit()

    # Add the project root to sys.path so imports from src work (already done above, but double check)
    if project_root not in sys.path:
        sys.path.append(project_root)
        print(f"Added project root to sys.path: {project_root}") # Optional print


    root = tk.Tk() # Create the main Tkinter window
    app = DRGuiApp(root) # Create an instance of the GUI application
    root.mainloop() # Start the Tkinter event loop (keeps the window open)

Added project root to sys.path: c:\Users\abdul\Downloads\ci-project\ci-project
Attempting to load data from: c:\Users\abdul\Downloads\ci-project\ci-project\data\diabetes_binary_5050split_health_indicators_BRFSS2015.csv
PCA fitted with 2 components.
Explained variance ratio: 0.2557
